# Use case 1: Molecular tracer diffusion in a human brain

## Introduction

In many medical studies, tracers are used to understand how solutes move through the human brain {cite}`rennels1985tracer`, {cite}`zhang1992rat`, {cite}`ni2013amyloid`.
They are preferred as one can use medical imaging techniques, such as {term}`PET` and {term}`MRI`, while {term}`CT` uses {term}`X-ray` images to construct a 3D model of an organ.

## Mathematical model

We start by describing the model of cerebrospinal fluid (CSF), based on {cite}`Causemann2025`.
This network consist of two connected regions, namely the ventricular system $\Omega_V$ and the subarachnoid space $\Omega_{SAS}$.
The subarachnoid space is the space between the membrane of the brain (pia matter) and the membrane towards the skull (dura matter).
The ventrical system is a cavity within the brain where cells produce CSF.
The CSF enters the brain at the top of the spinal coord. We will call this boundary $\Gamma_{SSAS}$ (Spinal subarachnoid space).
We consider the brain paraenchyma (the brain tissue) as being surrounded by CSF.
The interface between the parenchyma and the CSF can be split into two regions, the surface of the lateral ventricles $\Gamma_{LV}$ and the pial surface $\Gamma_{pia}$.
We consider no fluid flow through the pial surface. However, the ventricles produce fluid, and therefore we consider an influx through $\Gamma_{LV}$.
The CSF is surrounded by dura matter, which is denoted as $\Gamma_{AM}$. This region is subdivided into an upper ($\Gamma_{AM-U}$) and lower region ($\Gamma_{AM-L}$).
Fluid can penetrate the the upper region (and exit the system), while no flow occurs through the lower region.
Summarizing, we can write the fluid equations as

$$
\begin{align}
\mu \Delta \mathbf{u} + \nabla p &= 0 &&\text{in } \Omega_{CSF}\\
\nabla \cdot \mathbf{u} &=0 && \text{in } \Omega_{CSF}\\
\mathbf{u} &= 0 &&\text{in } \Omega_{PAR}\\
-\mu \left((\nabla \mathbf{u})\cdot\mathbf{n} + p \mathbf{n}\right)\cdot \mathbf{n}&= R_0(\mathbf{u}\cdot \mathbf{n}) && \text{on } \Gamma_{AM-U}\\
\mathbf{u} \cdot \mathbf{t} &=0 &&\text{on }\Gamma_{AM-U}\\
\mathbf{u}&= 0 &&\text{on } \Gamma_{AM-L}\cup\Gamma_{pia}\cup\Gamma_{SSAS}\\
\mathbf{u}\cdot \mathbf{n} &=\frac{1}{\vert\Gamma_{LV}\vert}u_{in} && \text{on } \Gamma_{LV}\\
\mathbf{u}\cdot\mathbf{t} &= 0 && \text{on } \Gamma_{LV}
\end{align}
$$


We solve the set of linear equation with the {term}`FEM` and the following variational form:

Find $u\in H_g^1(\mathrm{div}, \Omega_{\mathrm{CSF}})$, $p\in L^2(\Omega_{\mathrm{CSF}})$ such that

$$
a(u, p, v, q) = L(v, q) \quad \forall v\in H_0^1(\mathrm{div}, \Omega_{\mathrm{CSF}}), q\in L^2(\Omega_{\mathrm{CSF}})
$$

where

$$
\begin{align*}
a(u, p, v, q)&= \int_{\Omega_{CSF}} \mu \nabla \mathbf{u} \cdot \nabla \mathbf{v} - \nabla \cdot \mathbf{v} p ~\mathrm{d}x +  \int_{\Gamma_{AM-u}} R_0 \mathbf{u}\cdot \mathbf{n} \mathbf{n} \cdot\mathbf{v} ~\mathrm{d}s+\\
&+\int_{\mathcal{F}} -\mu (\{\nabla \mathbf{u}\}\cdot\mathbf{n}_F) \cdot \left[\mathbf{v}_t\right] -\mu (\{\nabla \mathbf{v}\}\cdot \mathbf{n}_F) \cdot \left[\mathbf{u}_t\right] + \frac{\sigma \mu}{h_F}\left[\mathbf{u}_t\right]~\mathrm{d}s\\
&+\int_{\Omega_{CSF}} \nabla\cdot \mathbf{u}q~\mathrm{d} x = 0.
\end{align*}
$$

and

$$
L(v, q) = 0.
$$


## References

```{bibliography}
:filter: docname in docnames
```
